In [ ]:
import openai

client = openai.OpenAI()

PROMPT = """
I have the following functions in my system.

'get_weather'
'get_currency'
'get_news'

All of them receive the name of a country as an argument (i.e. get_weather('Spain'))

Please, answer with the name of the function that you would like me to run.

Please, say nothing else, just the name of the function with the arguments.

Answer the following question:

What is the weather in Korea?
"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": PROMPT},
    ],
)
print(response.choices[0].message.content)

In [ ]:
import openai

clientForLoop = openai.OpenAI()
messages = []

def call_ai():
    response = clientForLoop.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": message})
    print("AI:", message)

while True:
    message = input("Send a message to the LLM...")
    if message == "quit":
        break
    else:
        messages.append({"role": "user", "content": message})
        print("User:", message)
        answer = call_ai()

In [ ]:
import openai
from openai.types.chat import ChatCompletionMessage
import json

def get_weather(city):
    return f"The weather in {city} is 25 degrees Celsius with clear skies."

FUNCTION_MAP = {
    'get_weather': get_weather,
}

TOOL = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather in a country",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of city to get the weather for",
                    }
                },
                "required": ["city"],
            },
        }
    }
]

clientForTool = openai.OpenAI()
messages = []

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls is not None and len(message.tool_calls) > 0:
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments,
                    }
                } for tool_call in message.tool_calls
            ]
        })

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function {function_name} with arguments {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
                print("Error decoding JSON arguments:", arguments)
            
            function_to_call = FUNCTION_MAP[function_name]
            function_response = function_to_call(**arguments)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": function_response,
            })
            print(f"Function response: {function_response}")
            call_ai_with_tool()
    else:
        messages.append({"role": "assistant", "content": message.content or ""})
        print("AI:", message.content or "")


    

def call_ai_with_tool():
    response = clientForTool.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOL,
    )
    message = response.choices[0].message
    process_ai_response(message)


while True:
    message = input("Send a message to the LLM...")
    if message == "quit":
        break
    else:
        messages.append({"role": "user", "content": message})
        print("User:", message)
        answer = call_ai_with_tool()

User: what is the weather of spain?
Calling function get_weather with arguments {"city":"Spain"}
Function response: The weather in Spain is 25 degrees Celsius with clear skies.
AI: The weather in Spain is currently 25 degrees Celsius with clear skies.
AI: ChatCompletionMessage(content='The weather in Spain is currently 25 degrees Celsius with clear skies.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
AI: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_vc9t8L5cIAjcxfsmEF29nnet', function=Function(arguments='{"city":"Spain"}', name='get_weather'), type='function')])
User: tell me what is the weather of spain?
AI: The weather in Spain is currently 25 degrees Celsius with clear skies.
AI: ChatCompletionMessage(content='The weather in Spain is currently 25 degrees Celsius with clear skies.', refusal=None, role='as